In [1]:
import javabridge
import os
import glob
import pandas as pd
import pydot
from IPython.display import SVG

In [2]:
tetrad_libdir = os.path.join(os.getcwd(), 'temp','py-causal','src', 'pycausal', 'lib')
for l in glob.glob(tetrad_libdir + os.sep + "*.jar"):
    print l
    javabridge.JARS.append(str(l))

/home/jovyan/work/temp/py-causal/src/pycausal/lib/tetrad-lib-6.0.1-SNAPSHOT-tetradcmd.jar


In [3]:
javabridge.start_vm(run_headless=True, max_heap_size = '100M')
javabridge.attach()

In [4]:
data_dir = os.path.join(os.getcwd(), 'temp','py-causal','data', 'charity.txt')
data_dir

'/home/jovyan/work/temp/py-causal/data/charity.txt'

In [5]:
dframe = pd.read_table(data_dir, sep="\t")
dframe.head()

,TangibilityCondition,AmountDonated,Sympathy,Imaginability,Impact
0,1,0,7,4,4.5
1,1,0,2,2,2.0
2,1,0,7,3,3.0
3,1,0,4,3,4.5
4,1,0,3,7,1.0


In [6]:
node_list = javabridge.JClassWrapper("java.util.ArrayList")()
# load dataset
for col in dframe.columns:
    nodi = javabridge.JClassWrapper("edu.cmu.tetrad.data.ContinuousVariable") (col)
    node_list.add(nodi)

In [7]:
dataBox = javabridge.JClassWrapper("edu.cmu.tetrad.data.DoubleDataBox")(len(dframe.index),dframe.columns.size)

In [8]:
for col in range(0,dframe.columns.size):
    for row in dframe.index:
        value = javabridge.JClassWrapper("java.lang.Double")(dframe.ix[row][col])
        dataBox.set(row,col,value)

In [9]:
boxData = javabridge.JClassWrapper("edu.cmu.tetrad.data.BoxDataSet")(dataBox, node_list)

In [10]:
tetradData = javabridge.JClassWrapper("edu.cmu.tetrad.data.CovarianceMatrixOnTheFly")(boxData)

In [11]:
alpha = 0.05
IndTest = javabridge.JClassWrapper('edu.cmu.tetrad.search.IndTestFisherZ')(tetradData, alpha)

In [12]:
rfci = javabridge.JClassWrapper('edu.cmu.tetrad.search.Rfci')(IndTest)

In [13]:
rfci.setDepth(-1)
rfci.setVerbose(True)

In [14]:
prior = javabridge.JClassWrapper('edu.cmu.tetrad.data.Knowledge2')()
prior.setForbidden('TangibilityCondition', 'Impact') # forbidden directed edges
prior.setRequired('Sympathy','TangibilityCondition') # required directed edges
prior.setTierForbiddenWithin(0, True)
prior.addToTier(0, 'TangibilityCondition')
prior.addToTier(0, 'Imaginability')
prior.addToTier(1, 'Sympathy')
prior.addToTier(1, 'AmountDonated')
prior.addToTier(2, 'Impact')
rfci.setKnowledge(prior)
prior

Instance of edu.cmu.tetrad.data.Knowledge2: /knowledge
addtemporal

0* TangibilityCondition Imaginability 
1 Sympathy AmountDonated 
2 Impact 

forbiddirect

TangibilityCondition ==> Impact 
requiredirect

Sympathy ==> TangibilityCondition 

In [15]:
tetradGraph = rfci.search()
tetradGraph

Instance of edu.cmu.tetrad.graph.EdgeListGraphSingleConnections: Graph Nodes:
TangibilityCondition,AmountDonated,Sympathy,Imaginability,Impact

Graph Edges:
1. AmountDonated o-> Impact
2. Imaginability o-> Impact
3. Imaginability o-> Sympathy
4. Sympathy --> AmountDonated
5. Sympathy o-> Impact
6. Sympathy --> TangibilityCondition

In [16]:
tetradGraph.toString()

u'Graph Nodes:\nTangibilityCondition,AmountDonated,Sympathy,Imaginability,Impact\n\nGraph Edges:\n1. AmountDonated o-> Impact\n2. Imaginability o-> Impact\n3. Imaginability o-> Sympathy\n4. Sympathy --> AmountDonated\n5. Sympathy o-> Impact\n6. Sympathy --> TangibilityCondition\n'

In [17]:
tetradGraph.getNodeNames()

Instance of java.util.ArrayList: [TangibilityCondition, AmountDonated, Sympathy, Imaginability, Impact]

In [18]:
tetradGraph.getEdges()

Instance of java.util.HashSet: [AmountDonated o-> Impact, Sympathy --> TangibilityCondition, Sympathy --> AmountDonated, Sympathy o-> Impact, Imaginability o-> Sympathy, Imaginability o-> Impact]

In [19]:
javabridge.detach()
javabridge.kill_vm()